## 1st assignment

We want the hours of service for each unique (stop_id, route_id) pair.  Please submit the Python code and your output in the following format:
 - Stop_id, route_id, earliest arrival time, latest  departure time, hours of service, Monday.
 - Please also provide the grand total of the hours of service (for all stops and routes) at the top of the sheet.


In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta 

In [2]:
#import calendar.txt
cal_df = pd.read_table('./calendar.txt', sep=",", header=0, names=['service_id', 'monday', 'tuesday',\
                                                                   'wednesday','thursday','friday'\
                                                                   ,'saturday','sunday','start_date','end_date'])

In [3]:
cal_df.head()

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,1,0,0,0,0,0,1,0,20111113,20120114
1,2,0,0,0,0,0,0,1,20111113,20120114
2,3,1,1,1,1,1,0,0,20111113,20120114
3,4,0,0,0,0,0,1,0,20111113,20120114
4,5,0,0,0,0,0,0,1,20111113,20120114


In [4]:
#import route.txt
route_df = pd.read_table('./routes.txt', sep=",", header=0, names=['route_id','route_short_name',\
                                                                   'route_long_name','route_type','route_url'])

In [5]:
route_df.head()

,route_id,route_short_name,route_long_name,route_type,route_url
0,37692,665,Darien-Westmont,3,http://www.pacebus.com/sub/schedules/route_det...
1,37571,821,IL 53/83rd St.-Belmont Feeder,3,http://www.pacebus.com/sub/schedules/route_det...
2,37536,663,Darien-Clarendon Hills,3,http://www.pacebus.com/sub/schedules/route_det...
3,37523,664,Willowbrook-Clarendon Hills,3,http://www.pacebus.com/sub/schedules/route_det...
4,37563,662,South Central Westmont Feeder,3,http://www.pacebus.com/sub/schedules/route_det...


In [6]:
#import stoptime.txt
stoptime_df = pd.read_table('./stop_times.txt', sep=",", header=0, names=['trip_id','arrival_time','departure_time'\
                                                                          ,'stop_id','stop_sequence'])

In [7]:
stoptime_df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,1897634,05:30:00,05:30:00,34922,2
1,1897634,05:30:00,05:30:00,34923,3
2,1897634,05:30:00,05:30:00,34924,4
3,1897634,05:30:00,05:30:00,34925,5
4,1897634,05:32:00,05:32:00,34926,6


In [8]:
stoptime_df.shape

(876878, 5)

In [9]:
#import trips.txt
trips_df = pd.read_table('./trips.txt', sep=",", header=0, names=['route_id','trip_id','service_id',\
                                                                  'direction_id','block_id'])

In [10]:
trips_df.head()

,route_id,trip_id,service_id,direction_id,block_id
0,37671,1900289,32,1,289671
1,37566,1900283,31,1,289693
2,37567,1906009,29,1,289799
3,37504,1900394,32,0,289710
4,37566,1900493,31,0,289717


## calculate hours of service
----
- stoptime(stop_id, arrival_time, departure_time, trip_id) + cal_df(service_id, monday)+ trips(trip_id,service_id,route_id)
     
     -> get modified_df (service_id	monday	route_id	trip_id	arrival_time	departure_time	stop_id	stop_sequence)
 
-  for final_table (stop_id, route_id, earliest_arrival_time, last_departure_time, hours of service, Monday)

In [11]:
modified_df = pd.merge(cal_df[['service_id','monday']], trips_df, on='service_id')

In [12]:
modified_df.head()

,service_id,monday,route_id,trip_id,direction_id,block_id
0,24,0,37659,1902411,1,290181
1,24,0,37643,1900791,1,290373
2,24,0,37671,1900353,0,289680
3,24,0,37653,1902414,1,290181
4,24,0,37643,1900787,1,290368


In [13]:
modified_df = pd.merge(modified_df[['service_id','monday','route_id','trip_id']],stoptime_df, on='trip_id') 

In [14]:
modified_df.sort_values(by=['arrival_time'],ascending=True).reset_index(drop=True).head()

,service_id,monday,route_id,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,32,1,37609,1903912,01:40:00,01:40:00,5761,2
1,32,1,37609,1903913,01:55:00,01:55:00,5761,2
2,32,1,37609,1903912,02:03:00,02:03:00,35138,3
3,32,1,37609,1903912,02:13:00,02:13:00,5797,4
4,32,1,37609,1903912,02:13:00,02:13:00,4736,5


In [15]:
modified_df.sort_values(by=['route_id','stop_id'],ascending=True).reset_index(drop=True).head()

,service_id,monday,route_id,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,28,0,37493,1907524,08:13:00,08:13:00,6269,400
1,28,0,37493,1907548,20:09:00,20:09:00,6269,1996
2,28,0,37493,1907537,14:25:00,14:25:00,6269,1199
3,28,0,37493,1907545,18:25:00,18:25:00,6269,1731
4,28,0,37493,1907535,13:25:00,13:25:00,6269,1066


In [16]:
# finding time above 24hour format
def check24hour(dep_time):
    flag=0
    time = dep_time.split(':')
    if (int(time[0])>=24):
        time = '23:59:59'
        flag=1
    else:
        time = ':'.join(time)
    return (time,flag);

# calculate hours of service by (last_time - earliest_time)
def timediff(earliest_time, last_time):
    arr = check24hour(earliest_time)
    arr_time = datetime.datetime.strptime(arr[0], '%H:%M:%S')
    dep = check24hour(last_time)
    dep_time = datetime.datetime.strptime(dep[0], '%H:%M:%S')
    diff = dep_time-arr_time
    if(arr[0]>dep[0]):
        diff = timedelta(seconds=0)
    elif(dep[1]== 1 and arr[1]!=1):
        diff+=timedelta(seconds=1)
    
    return diff

In [17]:
# finding unique (route_id, stop_id)
pair = modified_df[['route_id','stop_id']].values

x_hashable = map(tuple, pair)
keypair = set(x_hashable)
keypair = map(list,keypair)
keypair  = list(keypair)

print(np.array(keypair).shape)

(30135, 2)


In [28]:
final_list = []
sum_of_hours = timedelta(seconds=0)
for i in range(0,np.array(keypair).shape[0]):
    route_id = keypair[i][0]
    stop_id = keypair[i][1]
    spec = modified_df.loc[(modified_df['route_id'] == route_id) & (modified_df['stop_id'] == stop_id)]
    spec = spec.sort_values(by=['service_id','arrival_time'],ascending=True).reset_index(drop=True)
    spec = spec[['service_id','monday','route_id','stop_id', 'trip_id', 'arrival_time', 'departure_time']]

    #check if the service_id has service on monday
    #delete the rows which are not service on monday 
    #find the minimum for hte earlies_time and maximum for last_time
    service = 0
    if ((spec['monday']==1).any()) == True:
        service = 1
        spec = spec[spec.monday != 0]
        arrival = spec['arrival_time'].min()
        depart = spec['departure_time'].max()
        ser_hour = timediff(arrival, depart)
        sum_of_hours+= ser_hour
        final_list.append([stop_id,route_id,arrival, depart, str(ser_hour), service])
    else:
        arrival = spec['arrival_time'].min()
        depart = spec['departure_time'].max()
        final_list.append([stop_id,route_id,arrival, depart, str(timedelta(seconds=0)), service])

## Final_Sheet

In [29]:
# output final_list to final DataFrame
final = pd.DataFrame(final_list, columns = ['stop_id', 'route_id', 'earliest_arrival', 'last_departure',\
                                       'hours_of_service','monday'])
final = final.sort_values(by=['stop_id','route_id'],ascending=True).reset_index(drop=True)
final.head()

,stop_id,route_id,earliest_arrival,last_departure,hours_of_service,monday
0,104,37574,05:23:00,24:11:00,18:37:00,1
1,105,37574,05:23:00,24:11:00,18:37:00,1
2,106,37574,05:23:00,24:11:00,18:37:00,1
3,173,37504,26:12:00,26:12:00,0:00:00,1
4,173,37638,05:22:00,21:32:00,16:10:00,1


In [30]:
print('Grand Total of Hours of Service(for all stops and routes):',str(sum_of_hours))

Grand Total of Hours of Service(for all stops and routes): 13370 days, 20:29:00


In [33]:
final.head()

,stop_id,route_id,earliest_arrival,last_departure,hours_of_service,monday
0,104,37574,05:23:00,24:11:00,18:37:00,1
1,105,37574,05:23:00,24:11:00,18:37:00,1
2,106,37574,05:23:00,24:11:00,18:37:00,1
3,173,37504,26:12:00,26:12:00,0:00:00,1
4,173,37638,05:22:00,21:32:00,16:10:00,1


In [32]:
#export final DataFrame to excel_sheet
writer = pd.ExcelWriter('Assignment1.xlsx', engine='xlsxwriter')
text = 'Grand Total of Hours of Service(for all stops and routes): '+str(sum_of_hours)
final.to_excel(writer, startrow=1, startcol=0)
worksheet = writer.sheets['Sheet1']
worksheet.write(0, 0, text)
writer.save()